In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import expr, col, broadcast

spark = SparkSession.builder.appName("spark_job").config("spark.driver.memory", "12g").getOrCreate()

24/12/14 08:51:30 WARN SparkSession: Using an existing Spark session; only runtime SQL configurations will take effect.


In [2]:
spark.conf.set("spark.sql.autoBroadcastJoinThreshold", "-1")
spark.conf.set("spark.sql.debug.maxToStringFields", "-1")

In [3]:
match_details = spark.read.option("header", "true") \
            .csv("/home/iceberg/data/match_details.csv")

match_details.write.bucketBy(16, "match_id").mode("overwrite").saveAsTable("bootcamp.bucketed_match_details")

24/12/14 08:51:35 WARN SparkStringUtils: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


In [4]:
maps = spark.read.option("header", "true") \
            .option("inferSchema", "true") \
            .csv("/home/iceberg/data/maps.csv")

In [5]:
medals = spark.read.option("header", "true") \
            .option("inferSchema", "true") \
            .csv("/home/iceberg/data/medals.csv")

In [6]:
matches = spark.read.option("header", "true") \
            .csv("/home/iceberg/data/matches.csv")

matches.write.bucketBy(16, "match_id").mode("overwrite").saveAsTable("bootcamp.bucketed_matches")

In [ ]:
medals_matches_players = spark.read.option("header", "true") \
            .csv("/home/iceberg/data/medals_matches_players.csv")

medals_matches_players.write.bucketBy(16, "match_id").mode("overwrite").saveAsTable("bootcamp.bucketed_medals_matches_players")

In [3]:
bucketed_join = spark.sql("""
    SELECT * FROM bootcamp.bucketed_matches
        JOIN bootcamp.bucketed_match_details
            USING(match_id)
        JOIN bootcamp.bucketed_medals_matches_players
            USING(match_id)
""")

In [ ]:
bucketed_join.show(5)

ERROR:root:Exception while sending command.
Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 516, in send_command
    raise Py4JNetworkError("Answer from Java side is empty")
py4j.protocol.Py4JNetworkError: Answer from Java side is empty

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/opt/spark/python/lib/py4j-0.10.9.7-src.zip/py4j/clientserver.py", line 539, in send_command
    raise Py4JNetworkError(
py4j.protocol.Py4JNetworkError: Error while sending or receiving


In [8]:
explicit_medal_maps = medals.join(broadcast(maps))

In [10]:
explicit_medal_maps.show(5)

+----------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----+----------+--------------------+-------------------+--------------------+
|  medal_id|sprite_uri|sprite_left|sprite_top|sprite_sheet_width|sprite_sheet_height|sprite_width|sprite_height|classification|description|name|difficulty|               mapid|               name|         description|
+----------+----------+-----------+----------+------------------+-------------------+------------+-------------+--------------+-----------+----+----------+--------------------+-------------------+--------------------+
|2315448068|      NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|          NULL|       NULL|NULL|      NULL|c93d708f-f206-11e...|              Urban|Andesia was the c...|
|2315448068|      NULL|       NULL|      NULL|              NULL|               NULL|        NULL|         NULL|          NULL| 

In [ ]:
bucketMatches = matches.join(